# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy

import matplotlib.pyplot as plt
import requests

# Import API key
from api_keys_krempa import geoapify_key

In [ ]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

In [ ]:
city_data_df.info()

In [ ]:
df= city_data_df.dropna(how="any").reset_index(drop=True)
df.info()

---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_1 = df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot_1

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [ ]:
# Narrow down cities that fit criteria and drop any results with null values
mask = (df["Max Temp"] > 65) & (df["Max Temp"] < 80) & (df["Wind Speed"] < 10) & (df["Cloudiness"]==0)

# Drop any rows with null values
df2= df.loc[mask]

# Display sample data
df2

### Step 3: Create a new DataFrame called `hotel_df`.

### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [ ]:
latitude = -17.8000
longitude = -63.1667

# Set base URL
base_url = "https://api.geoapify.com/v2/places"


# Set the parameters for the type of place
categories = "acommodation.hotel"
radius = 10000

# Set the parameters for the type of search
filters = f"circle:{longitude},{latitude},{radius}"
bias = f"proximity:{longitude},{latitude}"
limit = 20

# set up a parameters dictionary
params = {
    "categories":categories,
    "limit":limit,
    "filter": filters,
    "bias":bias,
    "apiKey":geoapify_key    
}

response = requests.get(base_url, params=params)

status_code = response.status_code

if status_code == 200:
    data = response.json()
else:
    data = {}

results = data.get("features", [])

if len(results) > 0:
    result = results[0]
else:
    result = {}

properties = result.get("properties", {})

state = properties.get("state")
country = properties.get("country")
city = properties.get("city")
lon = properties.get("lon")
lat = properties.get("lat")
address = properties.get("formatted")
distance = properties.get("distance")

row = {
    "address" : address,
    "state" : state, 
    "country" : country,
    "city" : city, 
    "longitude" : lon, 
    "latitude" : lat,
    "distance": distance
}

row    

In [ ]:
rows = []

for index, row in df2.iterrows():
    latitude = row.Lat
    longitude = row.Lng
    humidity = row.Humidity
    orig_city = row.City

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Set the parameters for the type of place
    categories = "acommodation.hotel"
    radius = 10000

    # Set the parameters for the type of search
    filters = f"circle:{longitude},{latitude},{radius}"
    bias = f"proximity:{longitude},{latitude}"
    limit = 20

    # set up a parameters dictionary
    params = {
        "categories":categories,
        "limit":limit,
        "filter": filters,
        "bias":bias,
        "apiKey":geoapify_key    
    }

    response = requests.get(base_url, params=params)

    status_code = response.status_code

    if status_code == 200:
        data = response.json()
    else:
        data = {}

    results = data.get("features", [])

    if len(results) > 0:
        result = results[0]
    else:
        result = {}

    properties = result.get("properties", {})

    state = properties.get("state")
    country = properties.get("country")
    city = properties.get("city")
    address = properties.get("formatted")
    distance = properties.get("distance")
    name = properties.get("name")

    row = {
        "original_city" : orig_city,
        "longitude" : longitude, 
        "latitude" : latitude,
        "humidity" : humidity,
        "hotel": name,
        "address" : address,
        "state" : state, 
        "country" : country,
        "city" : city, 
        "distance": distance
    }

    print(orig_city)
    
rows.append(row)

In [ ]:
hotel_df = pd.DataFrame(rows)
hotel_df

In [ ]:
hotel_df = hotel_df.dropna(how="any").reset_index(drop=True)

### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [ ]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500, 
    size = "humidity",
    color = "city",
    hover_cols = ["address"]
)

# Display the map
map_plot_2